In [21]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_ibm import ChatWatsonx, WatsonxEmbeddings, WatsonxLLM

from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

In [49]:
model_id="ibm/granite-3-8b-instruct"
url="https://us-south.ml.cloud.ibm.com"
apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"
project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"

parameters = {
    "temperature": 0.0,
    "max_tokens": 1000,
}

model = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=apikey,
    project_id=project_id,
    params=parameters,
)

tavily_api_key = "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"

In [50]:
tavilySearchAPIWrapper = TavilySearchAPIWrapper(tavily_api_key=tavily_api_key)
search = TavilySearchResults(api_wrapper=tavilySearchAPIWrapper)

tool = TavilySearchResults(api_wrapper=tavilySearchAPIWrapper,  max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [51]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [52]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [53]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

abot = Agent(model, [tool], system=prompt)

In [54]:
messages = [HumanMessage(content="What is the weather in charlotte NC?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in charlotte NC'}, 'id': 'chatcmpl-tool-9a6e12b6e2f1482dabe6d4eefc3b23d2', 'type': 'tool_call'}
Back to the model!


In [55]:
result

{'messages': [HumanMessage(content='What is the weather in charlotte NC?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-9a6e12b6e2f1482dabe6d4eefc3b23d2', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in charlotte NC"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 224, 'total_tokens': 258}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chatcmpl-8c9962f10f68e042ca2d2a0a6a796228', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in charlotte NC'}, 'id': 'chatcmpl-tool-9a6e12b6e2f1482dabe6d4eefc3b23d2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 224, 'output_tokens': 34, 'total_tokens': 258}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Charlotte\', \'reg

In [56]:
result['messages'][-1].content

'The current weather in Charlotte, NC is partly cloudy with a temperature of 46.9°F (8.3°C). The wind is coming from the ESE at 4.0 mph (6.5 kph). There is no precipitation and the humidity is 40%. The weather forecast for the next few days suggests a mix of sunny and cloudy conditions with temperatures ranging from 43°F to 55°F. There is a possibility of rain on some days.'

In [31]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'chatcmpl-tool-d71123c42607404280f44f5990449c6d', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'chatcmpl-tool-16ccdbc32d65420d9893b36f847c8f4e', 'type': 'tool_call'}
Back to the model!


In [32]:
result['messages'][-1].content

'The weather in San Francisco is currently partly cloudy with a temperature of 59°F (15.0°C). The wind is blowing at 5.1 mph (8.3 kph) from the west-northwest. There is no precipitation and the humidity is 60%.\n\nIn Los Angeles, the weather is sunny with a temperature of 62.1°F (16.7°C). The wind is blowing at 7.6 mph (12.2 kph) from the west-southwest. There is no precipitation and the humidity is 62%.'

In [45]:
'''Supported models: ['codellama/codellama-34b-instruct-hf', 'google/flan-t5-xl', 'google/flan-t5-xxl', 'google/flan-ul2', 'ibm/granite-13b-instruct-v2', 'ibm/granite-20b-code-instruct', 'ibm/granite-20b-multilingual', 'ibm/granite-3-2-8b-instruct-preview-rc', 'ibm/granite-3-2b-instruct', 'ibm/granite-3-8b-instruct', 'ibm/granite-34b-code-instruct', 'ibm/granite-3b-code-instruct', 'ibm/granite-8b-code-instruct', 'ibm/granite-guardian-3-2b', 'ibm/granite-guardian-3-8b', 'meta-llama/llama-2-13b-chat', 'meta-llama/llama-3-1-70b-instruct', 'meta-llama/llama-3-1-8b-instruct', 'meta-llama/llama-3-2-11b-vision-instruct', 'meta-llama/llama-3-2-1b-instruct', 'meta-llama/llama-3-2-3b-instruct', 'meta-llama/llama-3-2-90b-vision-instruct', 'meta-llama/llama-3-3-70b-instruct', 'meta-llama/llama-3-405b-instruct', 'meta-llama/llama-guard-3-11b-vision', 'mistralai/mistral-large', 'mistralai/mixtral-8x7b-instruct-v01']'''


model_id="meta-llama/llama-3-3-70b-instruct"
url="https://us-south.ml.cloud.ibm.com"
apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"
project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"

parameters = {
    "temperature": 0.0,
    "max_tokens": 1000,
}

model = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=apikey,
    project_id=project_id,
    params=parameters,
)


In [46]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'super bowl 2024 winner'}, 'id': 'chatcmpl-tool-73a79040ce1f422cb6e5b277895024ac', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'chatcmpl-tool-420138d288c1490784564eb1447b1890', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP'}, 'id': 'chatcmpl-tool-76c134fa2edf4111a215ad9db3035b1e', 'type': 'tool_call'}
Back to the model!


In [47]:
print(result['messages'][-1].content)

The winner of the Super Bowl in 2024 was the Kansas City Chiefs. The Kansas City Chiefs' headquarters is located in the state of Missouri. The GDP of Missouri is $348.5 billion.


In [59]:
#!pip install tavily-python

In [60]:
from tavily import TavilyClient
client = TavilyClient(api_key=tavily_api_key)

In [68]:
# choose location (try to change to your own city!)

city = "Charlotte"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

#query = f"which team won the soccer world cup 1998? and in which country?"

In [69]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Charlotte', 'region': 'North Carolina', 'country': 'United States of America', 'lat': 35.2269, 'lon': -80.8433, 'tz_id': 'America/New_York', 'localtime_epoch': 1739931333, 'localtime': '2025-02-18 21:15'}, 'current': {'last_updated_epoch': 1739931300, 'last_updated': '2025-02-18 21:15', 'temp_c': 7.8, 'temp_f': 46.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 110, 'wind_dir': 'ESE', 'pressure_mb': 1023.0, 'pressure_in': 30.22, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 43, 'cloud': 75, 'feelslike_c': 6.8, 'feelslike_f': 44.3, 'windchill_c': 6.2, 'windchill_f': 43.2, 'heatindex_c': 7.3, 'heatindex_f': 45.2, 'dewpoint_c': 0.5, 'dewpoint_f': 32.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 7.6, 'gust_kph': 12.2}}


In [70]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "Charlotte",
        "region": "North Carolina",
        "country": "United States of America",
        "lat": 35.2269,
        "lon": -80.8433,
        "tz_id": "America/New_York",
        "localtime_epoch": 1739931333,
        "localtime": "2025-02-18 21:15"
    },
    "current": {
        "last_updated_epoch": 1739931300,
        "last_updated": "2025-02-18 21:15",
        "temp_c": 7.8,
        "temp_f": 46.0,
        "is_day": 0,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
            "code": 1003
        },
        "wind_mph": 4.0,
        "wind_kph": 6.5,
        "wind_degree": 110,
        "wind_dir": "ESE",
        "pressure_mb": 1023.0,
        "pressure_in": 30.22,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 43,
        "cloud": 75,
        "feelslike_c": 6.8,
        "feelslike_f": 44.3,
        "windchill_c": 6.2,
        "